In [25]:
"""
Learning to solve parametric Quadratic Programming
portfolio optimization

Problem formulation:
    minimize    - p^T x + lambda x^T Q x
    subject to       1^T x = 1
                      x >= 0

Where p is interpreted as a vector of asset returns, and Q represents
the covariance between assets, which forms a penalty on overall
covariance (risk) weighted by lambda.

A solution is first predicted by a neural network
then a final layer is added to apply a trainable optimization routine for improved solutions.
"""

'\nLearning to solve parametric Quadratic Programming\nportfolio optimization\n\nProblem formulation:\n    minimize    - p^T x + lambda x^T Q x\n    subject to       1^T x = 1\n                      x >= 0\n\nWhere p is interpreted as a vector of asset returns, and Q represents\nthe covariance between assets, which forms a penalty on overall\ncovariance (risk) weighted by lambda.\n\nA solution is first predicted by a neural network\nthen a final layer is added to apply a trainable optimization routine for improved solutions.\n'

In [26]:

import cvxpy as cp
import numpy as np
import time
import torch
import torch.nn as nn
import neuromancer.slim as slim
import matplotlib.pyplot as plt
import matplotlib.patheffects as patheffects

from neuromancer.trainer import Trainer
from neuromancer.problem import Problem
from neuromancer.constraint import variable
from neuromancer.dataset import DictDataset
from neuromancer.loss import PenaltyLoss
from neuromancer.modules import blocks
from neuromancer.system import Node

from portfolio_utils import gen_portfolio_lto_data, cvx_qp
import cvxpy as cp
from cvxpylayers.torch.cvxpylayer import CvxpyLayer


from LOPO import DRSolver
from LOPO import ADMMSolver
from LOPO import ParaMetricDiagonal



In [27]:

""
"""
# # #  Dataset
"""
data_seed = 408
np.random.seed(data_seed)
batsize = 100
n_dim = 105  #problem size
n_train = 500 #number of training examples
n_valid = 100 #number of validation examples
n_test = 100 # number of test examples
budget = 10 #This controls the equality constraint LHS, i.e. sum(x) = budget,

# create dictionaries with sampled datapoints with uniform distribution
#data_loaded = np.load('portfolio_data/portfolio_var50_ineq50_eq1_ex12000.npz', allow_pickle=True)
data_loaded = gen_portfolio_lto_data(n_dim,n_train,n_valid,n_test)
Q_load = data_loaded['Q']
A_load = data_loaded['A']
G_load = data_loaded['G']
h_load = data_loaded['h']
x_load = data_loaded['x']
p_train = data_loaded['trainX']
p_valid = data_loaded['validX']
p_test  = data_loaded['testX']
sols_train = data_loaded['trainY']
sols_valid = data_loaded['validY']
sols_test  = data_loaded['testY']
#feat_size_load = data_loaded['feat_size']
print("p_train.dtype")
print( p_train.dtype )
samples_train = {"p": torch.Tensor(p_train)}  # JK TODO fix this, reduced size for debugging
samples_dev   = {"p": torch.Tensor(p_valid)}
samples_test  = {"p": torch.Tensor(p_test )}
# create named dictionary datasets
train_data = DictDataset(samples_train, name='train')
dev_data   = DictDataset(samples_dev,   name='dev')
test_data  = DictDataset(samples_test,  name='test')
# create torch dataloaders for the Trainer
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batsize, num_workers=0,
                                            collate_fn=train_data.collate_fn, shuffle=True)
dev_loader   = torch.utils.data.DataLoader(dev_data, batch_size=batsize, num_workers=0,
                                            collate_fn=dev_data.collate_fn, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_data, batch_size=batsize, num_workers=0,
                                            collate_fn=test_data.collate_fn, shuffle=True)
# note: training quality will depend on the DataLoader parameters such as batch size and shuffle



Generating dataset . . . 
Training set generated 
Validation set generated 
Test set generated 
p_train.dtype
float64


In [28]:
"""
# primal solution map architecture
"""
# define neural architecture for the solution map
func = blocks.MLP(insize=n_dim, outsize=n_dim,
                bias=True,
                linear_map=slim.maps['linear'],
                nonlin=nn.ReLU,
                hsizes=[n_dim*2] * 4)
# define symbolic solution map with concatenated features (problem parameters)
#xi = lambda p1, p2: torch.cat([p1, p2], dim=-1)
#features = Node(xi, ['p1', 'p2'], ['xi'], name='features')
sol_map = Node(func, ['p'], ['x'], name='map')
# trainable components of the problem solution
components = [sol_map]

"""
# # # objective and constraints formulation in Neuromancer
"""
# variables
x = variable("x")

# sampled parameters
p = variable('p')
Q = torch.Tensor(Q_load)

# objective function
lambd = 1.0
f = torch.sum(-p*x, dim = 1) + torch.sum( x*torch.matmul(Q,x.T).T, dim=1 ) #-p@x + lambd * x@Q@x
obj = f.minimize(weight=1.0, name='obj')
objectives = [obj]

# constraints
e = torch.ones(n_dim)
Q_con = 100.
con_1 = Q_con*(torch.sum(x, dim=1) == budget) #Q_con*(e@x == 1)
con_1.name = 'c1'
con_2 = Q_con * (x >= 0)
con_2.name = 'c2'
constraints = [con_1, con_2]

"""
# # #  problem formulation in Neuromancer
"""
# create penalty method loss function
loss = PenaltyLoss(objectives, constraints)
# construct constrained optimization problem
problem = Problem(components, loss)



In [29]:
"""
# # #  problem solution in Neuromancer
"""
optimizer = torch.optim.AdamW(problem.parameters(), lr=1e-3)
# define trainer
trainer = Trainer(
    problem,
    train_loader,
    dev_loader,
    test_loader,
    optimizer,
    epochs=500,
    patience=100,
    warmup=100,
    train_metric="train_loss",
    dev_metric="dev_loss",
    test_metric="test_loss",
    eval_metric="dev_loss",
)
# Train solution map
best_model = trainer.train()

/Users/king339/Projects/DAIDIST/neuromancer/src/neuromancer/constraint.py:169: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(left, right)


epoch: 0  train_loss: 812.5565795898438
epoch: 1  train_loss: 286.09613037109375
epoch: 2  train_loss: 146.6123046875
epoch: 3  train_loss: 79.90568542480469
epoch: 4  train_loss: 51.112327575683594
epoch: 5  train_loss: 64.59877014160156
epoch: 6  train_loss: 30.662967681884766
epoch: 7  train_loss: 29.370946884155273
epoch: 8  train_loss: 50.18938446044922
epoch: 9  train_loss: 42.83493423461914
epoch: 10  train_loss: 37.08600997924805
epoch: 11  train_loss: 40.70547103881836
epoch: 12  train_loss: 22.015390396118164
epoch: 13  train_loss: 29.56963539123535
epoch: 14  train_loss: 25.970138549804688
epoch: 15  train_loss: 23.595111846923828
epoch: 16  train_loss: 23.977676391601562
epoch: 17  train_loss: 24.165555953979492
epoch: 18  train_loss: 13.786473274230957
epoch: 19  train_loss: 28.247879028320312
epoch: 20  train_loss: 40.62012481689453
epoch: 21  train_loss: 21.53067970275879
epoch: 22  train_loss: 16.725830078125
epoch: 23  train_loss: 15.045286178588867
epoch: 24  train_lo

In [48]:
'''
Add ON A CORRECTION LAYER
'''

'''
# DEFINE THE OBJECTIVE
'''
# x is assumed to include slack variables!
def f_obj(x,p):
    x = x[:n_dim]
    return -p@x + lambd * x@(Q@x) 

'''
# DEFINE THE CONSTRAINTS
'''
def F_ineq(x,p):
    x = x[:n_dim]
    return -x

def F_eq(x,p):
    x = x[:n_dim]
    return (x.sum() - budget).unsqueeze(0)

num_steps = 20 # number of solver iterations to take
x_dim = n_dim # dimension of primal variable
n_ineq = n_dim #number of inequality constraints
n_eq = 1 #number of equality constraints
parm_dim = n_dim #number of problem parameters

#metric scaling parameters
lb_P = 1.0/50.0
ub_P = 50.0
scl_lb_P = 0.05
scl_ub_P = 0.7
neq_dim = x_dim + n_ineq
metric = ParaMetricDiagonal(neq_dim,parm_dim,ub_P,lb_P,scl_upper_bound=scl_ub_P,scl_lower_bound=scl_lb_P)

solver = 'DR'
#solver = 'ADMM'

if solver == 'DR':
    solver = DRSolver(
        f_obj = f_obj, 
        F_ineq = F_ineq,
        F_eq = F_eq,
        x_dim = x_dim, 
        n_ineq = n_ineq,
        n_eq = n_eq, 
        JF_fixed=True,
        Hf_fixed = True,
        num_steps = num_steps,
        metric = metric
        )
    
if solver == 'ADMM':
    solver = ADMMSolver(
        f_obj = f_obj, 
        F_ineq = F_ineq,
        F_eq = F_eq,
        x_dim = x_dim, 
        n_ineq = n_ineq,
        n_eq = n_eq, 
        JF_fixed=True,
        Hf_fixed = True,
        num_steps = num_steps,
        metric = metric
        )


# REMAP THROUGH CORRECTION
sol_map = Node(func, ['p'], ['x_predicted'], name='map')
correction = Node(solver,['x_predicted','p'],['x','cnv_gap'])
components = [sol_map, correction]

### ADD A CONVERGENCE PENALTY
cnv_gap = variable("cnv_gap")
f_cnv = cnv_gap
cnv_obj = f_cnv.minimize(weight=1e8, name='cnv_obj')
objectives = [cnv_obj]
constraints = []

# create loss function
loss = PenaltyLoss(objectives, constraints)
# construct constrained optimization problem
problem = Problem(components, loss)

In [49]:
'''
Train a correction layer
'''
DR_train_epochs = 10
optimizer = torch.optim.AdamW(solver.parameters(), lr=1e-3)
# define trainer
trainer = Trainer(
    problem,
    train_loader,
    dev_loader,
    test_loader,
    optimizer,
    epochs=DR_train_epochs,
    patience=500,
    warmup=100,
    train_metric="train_loss",
    dev_metric="dev_loss",
    test_metric="test_loss",
    eval_metric="dev_loss",
)
# Train solution map
best_model = trainer.train()

epoch: 0  train_loss: 10.003763198852539
epoch: 1  train_loss: 3.6839916706085205
epoch: 2  train_loss: 0.42106127738952637
epoch: 3  train_loss: 0.09322969615459442
epoch: 4  train_loss: 0.08394786715507507
epoch: 5  train_loss: 0.06182097643613815
epoch: 6  train_loss: 0.043070219457149506
epoch: 7  train_loss: 0.030605733394622803
epoch: 8  train_loss: 0.021146198734641075
epoch: 9  train_loss: 0.018580913543701172


## Compare to CVXPY Solver

In [50]:

with torch.no_grad():
    samples_test['name'] = 'test'
    model_out = problem(samples_test)

x_nm_test = model_out['test_' + "x"].detach().numpy()
x_loaded_test  = data_loaded['testY']

cvxpy_layer = cvx_qp(n_dim,Q,budget)
x_cvxpy_test = cvxpy_layer(samples_test['p'])

sol_diff = np.mean(np.sum((x_cvxpy_test.detach().numpy() - x_nm_test[:,0:105])**2,axis=-1))
rel_sol_diff = np.mean(np.sum((x_cvxpy_test.detach().numpy() - x_nm_test[:,0:105])**2,axis=-1)/np.sum(x_cvxpy_test.detach().numpy()**2,axis = -1))
print("Average relative Solution Difference:",np.mean(rel_sol_diff))
print("Average Solution difference: ", np.mean(sol_diff))

Average relative Solution Difference: 2.159302e-08
Average Solution difference:  3.566367e-08
